# Prep Above/Below Data Notebook
fetch + process nyc underground (subway) + street level (citibike) data for visualization

In [15]:
# Setup and Configuration
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import numpy as np
from pathlib import Path

# create output directory
output_dir = Path('../visualizations/above_below_data')
output_dir.mkdir(exist_ok=True, parents=True)

print("ABOVE/BELOW DATA PREPARATION")
print("=" * 60)

ABOVE/BELOW DATA PREPARATION


In [16]:
# Fetch Subway Data (the underground)
print("\n1. FETCHING SUBWAY DATA (Underground Layer)")
print("-" * 40)

# get one week of jan 2024 for prototype
subway_url = "https://data.ny.gov/resource/wujg-7c2s.json"
subway_data = []

start_date = datetime(2024, 1, 15)  # monday
end_date = datetime(2024, 1, 21)    # sunday

for days in range(7):
    date = start_date + timedelta(days=days)
    date_str = date.strftime('%Y-%m-%d')
    
    params = {
        '$limit': 50000,
        '$where': f"transit_timestamp >= '{date_str}T00:00:00' AND transit_timestamp < '{date_str}T23:59:59'"
    }
    
    print(f"fetching subway data for {date_str}...")
    response = requests.get(subway_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        subway_data.extend(data)
        print(f"  got {len(data)} records")

subway_df = pd.DataFrame(subway_data)
subway_df['transit_timestamp'] = pd.to_datetime(subway_df['transit_timestamp'])
subway_df['hour'] = subway_df['transit_timestamp'].dt.hour
subway_df['ridership'] = pd.to_numeric(subway_df['ridership'], errors='coerce')

# aggregate by station and hour
subway_hourly = subway_df.groupby(['station_complex', 'hour', 'latitude', 'longitude']).agg({
    'ridership': 'sum'
}).reset_index()

print(f"\nsubway summary:")
print(f"  stations: {subway_df['station_complex'].nunique()}")
print(f"  total weekly ridership: {subway_df['ridership'].sum():,.0f}")


1. FETCHING SUBWAY DATA (Underground Layer)
----------------------------------------
fetching subway data for 2024-01-15...
  got 50000 records
fetching subway data for 2024-01-16...
  got 50000 records
fetching subway data for 2024-01-17...
  got 50000 records
fetching subway data for 2024-01-18...
  got 50000 records
fetching subway data for 2024-01-19...
  got 50000 records
fetching subway data for 2024-01-20...
  got 50000 records
fetching subway data for 2024-01-21...
  got 50000 records

subway summary:
  stations: 428
  total weekly ridership: 13,487,090


In [17]:
# Fetch CitiBike Data (street level)
print("\n2. FETCHING CITIBIKE DATA (Street Layer)")
print("-" * 40)

# citibike has real-time data, but also historical trips
citibike_stations_url = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"

print("fetching citibike station locations...")
response = requests.get(citibike_stations_url)
citibike_stations = response.json()['data']['stations']
print(f"  found {len(citibike_stations)} citibike stations")

# convert to dataframe
citibike_df = pd.DataFrame(citibike_stations)

# for historical trip data, need https://s3.amazonaws.com/tripdata/202401-citibike-tripdata.csv.zip
# but for prototype, simulating activity based on station capacity and location

# create simulated hourly patterns based on station characteristics
citibike_df['capacity'] = pd.to_numeric(citibike_df['capacity'], errors='coerce')


2. FETCHING CITIBIKE DATA (Street Layer)
----------------------------------------
fetching citibike station locations...
  found 2240 citibike stations


In [18]:
# Match Subway Stations with Nearby CitiBike Stations
print("\n3. FINDING SPATIAL RELATIONSHIPS")
print("-" * 40)

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """calculate distance between two points in meters"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return c * 6371000  # earth radius in meters

# find citibike stations near each subway station
subway_stations = subway_df[['station_complex', 'latitude', 'longitude']].drop_duplicates()
subway_stations['latitude'] = pd.to_numeric(subway_stations['latitude'])
subway_stations['longitude'] = pd.to_numeric(subway_stations['longitude'])

proximity_threshold = 500  # meters

connections = []
for _, subway in subway_stations.iterrows():
    nearby_bikes = []
    for _, bike in citibike_df.iterrows():
        distance = haversine(
            float(subway['longitude']), float(subway['latitude']),
            float(bike['lon']), float(bike['lat'])
        )
        if distance <= proximity_threshold:
            nearby_bikes.append({
                'subway_station': subway['station_complex'],
                'bike_station_id': bike['station_id'],
                'bike_station_name': bike['name'],
                'distance': distance
            })
    connections.extend(nearby_bikes)

connections_df = pd.DataFrame(connections)
print(f"found {len(connections_df)} subway-citibike connections within {proximity_threshold}m")


3. FINDING SPATIAL RELATIONSHIPS
----------------------------------------
found 3556 subway-citibike connections within 500m


In [19]:
# Fetch Weather Data (the mood modifier)
print("\n4. FETCHING WEATHER DATA")
print("-" * 40)

# use noaa central park data
weather_url = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2024/72505394728.csv"

print("fetching central park weather for january 2024...")
weather_df = pd.read_csv(weather_url)

# filter to week
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_week = weather_df[
    (weather_df['DATE'] >= start_date) & 
    (weather_df['DATE'] <= end_date)
].copy()

# extract key weather metrics
weather_week['temp_f'] = pd.to_numeric(weather_week['TEMP'], errors='coerce')
weather_week['precipitation'] = pd.to_numeric(weather_week['PRCP'], errors='coerce')
weather_week['wind_speed'] = pd.to_numeric(weather_week['WDSP'], errors='coerce')

print(f"weather summary for the week:")
for _, day in weather_week.iterrows():
    print(f"  {day['DATE'].strftime('%a %m/%d')}: {day['temp_f']:.1f}°F, precip: {day['precipitation']:.2f}\"")


4. FETCHING WEATHER DATA
----------------------------------------
fetching central park weather for january 2024...
weather summary for the week:
  Mon 01/15: 26.9°F, precip: 0.00"
  Tue 01/16: 27.9°F, precip: 0.13"
  Wed 01/17: 22.0°F, precip: 0.19"
  Thu 01/18: 26.0°F, precip: 0.00"
  Fri 01/19: 30.6°F, precip: 0.00"
  Sat 01/20: 22.4°F, precip: 0.04"
  Sun 01/21: 23.7°F, precip: 0.00"


In [20]:
# Fetch Art Stations (portals)
print("\n5. FETCHING ART STATIONS")
print("-" * 40)

art_url = "https://data.ny.gov/resource/4y8j-9pkd.json"
response = requests.get(art_url, params={'$limit': 1000})
art_data = response.json()
art_df = pd.DataFrame(art_data)

# use normalized matching from earlier
art_df['station_normalized'] = art_df['station_name'].str.lower().str.replace(r'\([^)]*\)', '', regex=True).str.strip()
subway_stations['station_normalized'] = subway_stations['station_complex'].str.lower().str.replace(r'\([^)]*\)', '', regex=True).str.strip()

# find matches
art_matches = pd.merge(
    art_df[['station_normalized', 'art_title', 'artist']],
    subway_stations[['station_normalized', 'station_complex', 'latitude', 'longitude']],
    on='station_normalized',
    how='inner'
)

print(f"found {len(art_matches)} art installations at {art_matches['station_complex'].nunique()} stations")
print("\nexample art stations (the portals):")
for station in art_matches['station_complex'].unique()[:5]:
    art_count = len(art_matches[art_matches['station_complex'] == station])
    print(f"  {station}: {art_count} artworks")


5. FETCHING ART STATIONS
----------------------------------------
found 466 art installations at 274 stations

example art stations (the portals):
  Clark St (2,3): 1 artworks
  125 St (A,C,B,D): 4 artworks
  125 St (1): 4 artworks
  125 St (4,5,6): 4 artworks
  125 St (2,3): 4 artworks


In [21]:
# Create Hourly Animation Data
print("\n6. CREATING HOURLY ANIMATION DATA")
print("-" * 40)

# create 168 frames (7 days × 24 hours)
animation_data = []

for day in range(7):
    date = start_date + timedelta(days=day)
    
    # get weather for this day
    day_weather = weather_week[weather_week['DATE'].dt.date == date.date()].iloc[0] if len(weather_week) > 0 else None
    
    for hour in range(24):
        timestamp = date + timedelta(hours=hour)
        
        # subway activity for this hour
        subway_hour = subway_hourly[subway_hourly['hour'] == hour].copy()
        
        # simulate citibike inverse pattern
        # when subway is busy, bikes are less used (people underground)
        # adjust for weather
        if day_weather is not None and float(day_weather['precipitation']) > 0.1:  # convert to float
            bike_multiplier = 0.3  # rain kills bike usage
            subway_multiplier = 1.5  # rain increases subway usage
        else:
            bike_multiplier = 1.0
            subway_multiplier = 1.0
        
        # create inverse relationship
        if hour in [7, 8, 9, 17, 18, 19]:  # rush hours
            bike_activity = 0.4 * bike_multiplier  # low bike use during rush
            subway_activity = 1.0 * subway_multiplier  # high subway use
        elif hour in [0, 1, 2, 3, 4]:  # late night
            bike_activity = 0.1 * bike_multiplier
            subway_activity = 0.1 * subway_multiplier
        else:  # midday
            bike_activity = 0.7 * bike_multiplier  # bikes popular midday
            subway_activity = 0.5 * subway_multiplier
        
        frame = {
            'timestamp': timestamp.isoformat(),
            'day': int(day),  # convert to native int
            'hour': int(hour),  # convert to native int
            'weather': {
                'temp': float(day_weather['temp_f']) if day_weather is not None else 50.0,
                'raining': bool(float(day_weather['precipitation']) > 0.1) if day_weather is not None else False,  # convert to bool
                'wind': float(day_weather['wind_speed']) if day_weather is not None else 5.0
            },
            'subway_activity': float(subway_activity),  # ensure float
            'bike_activity': float(bike_activity),  # ensure float
            'is_rush_hour': bool(hour in [7, 8, 9, 17, 18, 19]),  # explicit bool
            'is_weekend': bool(day >= 5)  # explicit bool
        }
        animation_data.append(frame)

print(f"created {len(animation_data)} hourly frames")


6. CREATING HOURLY ANIMATION DATA
----------------------------------------
created 168 hourly frames


In [22]:
#Export All Data for Visualization
print("\n7. EXPORTING DATA")
print("-" * 40)

# 1. subway stations with hourly patterns
subway_export = {
    'stations': [],
    'hourly_patterns': {}
}

for station in subway_stations['station_complex'].unique():
    station_data = subway_df[subway_df['station_complex'] == station].iloc[0]
    
    # get hourly pattern
    hourly = subway_hourly[subway_hourly['station_complex'] == station].groupby('hour')['ridership'].mean()
    
    # check if it's an art station
    is_art_station = station in art_matches['station_complex'].values
    
    subway_export['stations'].append({
        'name': str(station),  # ensure string
        'lat': float(station_data['latitude']),  # ensure float
        'lon': float(station_data['longitude']),  # ensure float
        'is_portal': bool(is_art_station),  # ensure bool
        'art_count': int(len(art_matches[art_matches['station_complex'] == station])) if is_art_station else 0  # ensure int
    })
    
    # convert hourly pattern to list of floats
    hourly_list = [float(x) for x in hourly.tolist()] if len(hourly) > 0 else [0.0] * 24
    subway_export['hourly_patterns'][str(station)] = hourly_list

# 2. citibike stations
citibike_export = {
    'stations': []
}

for _, station in citibike_df.iterrows():
    citibike_export['stations'].append({
        'id': str(station['station_id']),  # ensure string
        'name': str(station['name']),  # ensure string
        'lat': float(station['lat']),  # ensure float
        'lon': float(station['lon']),  # ensure float
        'capacity': int(station['capacity']) if pd.notna(station['capacity']) else 20  # ensure int
    })

# 3. connections between layers - ensure all are native types
connections_export = []
for _, conn in connections_df.iterrows():
    connections_export.append({
        'subway_station': str(conn['subway_station']),
        'bike_station_id': str(conn['bike_station_id']),
        'bike_station_name': str(conn['bike_station_name']),
        'distance': float(conn['distance'])
    })

# 4. save everything
with open(output_dir / 'subway_data.json', 'w') as f:
    json.dump(subway_export, f)
    print(f"saved subway data: {len(subway_export['stations'])} stations")

with open(output_dir / 'citibike_data.json', 'w') as f:
    json.dump(citibike_export, f)
    print(f"saved citibike data: {len(citibike_export['stations'])} stations")

with open(output_dir / 'connections.json', 'w') as f:
    json.dump(connections_export, f)
    print(f"saved connections: {len(connections_export)} links")

with open(output_dir / 'animation_timeline.json', 'w') as f:
    json.dump(animation_data, f)
    print(f"saved animation timeline: {len(animation_data)} frames")

print("\n" + "=" * 60)
print("DATA PREPARATION COMPLETE!")
print("ready to build the visualization")
print(f"output directory: {output_dir}")


7. EXPORTING DATA
----------------------------------------
saved subway data: 428 stations
saved citibike data: 2240 stations
saved connections: 3556 links
saved animation timeline: 168 frames

DATA PREPARATION COMPLETE!
ready to build the visualization
output directory: ../visualizations/above_below_data
